Importing Required Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import nltk
import re

Loading Dataset

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/IBM/datasets/spam.csv',encoding="ISO-8859-1")

**Data Preprocessing**

Encoding the categorical column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data['v1']=le.fit_transform(data['v1'])

In [ ]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,1,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,0,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,0,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,1,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,1,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
pe=PorterStemmer()
arr=[]

In [ ]:
data.shape

(5572, 5)

In [ ]:
for i in range(5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()

  msg=[pe.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  arr.append(msg)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000)

In [ ]:
x=cv.fit_transform(arr).toarray()

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
x.shape

(5572, 6221)

In [ ]:
y=data['v1'].values

In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

train and test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

**Model Building**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# creating model
model=Sequential()

In [ ]:
# adding layers
model.add(Dense(units=6221,activation='relu'))
model.add(Dense(units=10000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
# compiling model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# fitting model
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
140/140 [==============================] - 125s 873ms/step - loss: 0.1346 - accuracy: 0.9652
Epoch 2/10
140/140 [==============================] - 113s 806ms/step - loss: 0.0150 - accuracy: 0.9971
Epoch 3/10
140/140 [==============================] - 115s 820ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 4/10
140/140 [==============================] - 112s 803ms/step - loss: 2.3099e-04 - accuracy: 1.0000
Epoch 5/10
140/140 [==============================] - 117s 834ms/step - loss: 6.7979e-05 - accuracy: 1.0000
Epoch 6/10
140/140 [==============================] - 115s 822ms/step - loss: 3.5247e-05 - accuracy: 1.0000
Epoch 7/10
140/140 [==============================] - 116s 831ms/step - loss: 2.0920e-05 - accuracy: 1.0000
Epoch 8/10
140/140 [==============================] - 115s 823ms/step - loss: 1.3786e-05 - accuracy: 1.0000
Epoch 9/10
140/140 [==============================] - 113s 806ms/step - loss: 9.5653e-06 - accuracy: 1.0000
Epoch 10/10
140/140 [===================

Saving Model

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd /content/drive/MyDrive/IBM/datasets

/content/drive/MyDrive/IBM/datasets


In [ ]:
model.save('spamDetector.h5')

Testing Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model('/content/drive/MyDrive/IBM/datasets/spamDetector.h5')

In [ ]:
def preprocessing(text):
  text=re.sub('[^a-zA-Z]',' ',text)
  text=text.lower()
  text=text.split()
  text=[pe.stem(word) for word in text if not word in set(stopwords.words('english'))]
  text=' '.join(text)
  return text

In [ ]:
# spam checking

In [ ]:
message1='Get your free ringtone. ude the password XX2Y to activate.'

In [ ]:
message1=preprocessing(message)

In [ ]:
message1

'get free rington ude password xx activ'

In [ ]:
result=model.predict(cv.transform([message1]))

1/1 [==============================] - 0s 39ms/step


In [ ]:
result

array([[0.9997627]], dtype=float32)

In [ ]:
if(result>0.5):
  print("Spam Message !!!")
else:
  print("Not Spam")

Spam Message !!!


In [ ]:
# ham checking

In [ ]:
message2='Please, call me when you are free.'

In [ ]:
message2=preprocessing(message2)

In [ ]:
message2

'pleas call free'

In [ ]:
result=model.predict(cv.transform([message2]))

1/1 [==============================] - 0s 25ms/step


In [ ]:
result

array([[0.00128711]], dtype=float32)

In [ ]:
if(result>0.5):
  print("Spam Message !!!")
else:
  print("Not Spam")

Not Spam
